In [1]:
from kafka import KafkaConsumer
import json ,pg, datetime,requests

In [2]:
POWERBI_URL = "https://api.powerbi.com/beta/901cb4ca-b862-4029-9306-e5cd0f6d9f86/datasets/56687642-d495-4acc-8383-b0761ab10a85/rows?experience=power-bi&key=%2F%2Bw%2FczPLyO2NBFUyyWct2ozUmrHhMTrRtnA7FBuew4iaq4n%2B3seSLOr1Zrby6307Dxy0QccWb0TJihhFqTwilg%3D%3D"
con=pg.connect('redsa','localhost',5432,None,'mec14','pass@word123')
con.query("CREATE TABLE IF NOT EXISTS reviews(id serial primary key,id_review varchar(255),label varchar(255),value text,date timestamp)")


NOTICE:  relation "reviews" already exists, skipping


In [3]:
consumer = KafkaConsumer('stock-data', bootstrap_servers=['localhost:9092'],
    auto_offset_reset='earliest',  # Start from the earliest message
    enable_auto_commit=True,
    group_id='location-group',
    value_deserializer=lambda x: json.loads(x.decode('utf-8')))

def storedata():
    for data in consumer:
        resul=data.value
        result=con.query(f"select * from reviews where id_review='{resul['id']}'")
        if len(result.getresult())==0:
            con.query(f"INSERT INTO reviews (id_review,label,value,date) VALUES ('{resul['id']}','{resul['label']}','{resul['text'].replace("'","''")}','{datetime.datetime.fromtimestamp(resul['date'])}')")
            requests.post(POWERBI_URL, json=resul)
            print('data stored')
        else:
            print('data already exists')
        


In [6]:
storedata()

data already exists
data already exists
data already exists


KeyboardInterrupt: 